# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f48e0f5f940>
├── 'a' --> tensor([[ 0.0421, -1.0498,  0.2443],
│                   [-1.6149, -0.4498, -0.3010]])
└── 'x' --> <FastTreeValue 0x7f48e0f5f4f0>
    └── 'c' --> tensor([[-0.5066,  0.2429, -0.2976,  0.6565],
                        [-0.2780, -0.2700, -1.9849, -0.3109],
                        [ 0.2164, -0.5135, -1.0767,  2.2482]])

In [4]:
t.a

tensor([[ 0.0421, -1.0498,  0.2443],
        [-1.6149, -0.4498, -0.3010]])

In [5]:
%timeit t.a

81.1 ns ± 1.06 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f48e0f5f940>
├── 'a' --> tensor([[ 0.2154, -0.6978, -0.0261],
│                   [ 0.8041, -0.6789, -0.9458]])
└── 'x' --> <FastTreeValue 0x7f48e0f5f4f0>
    └── 'c' --> tensor([[-0.5066,  0.2429, -0.2976,  0.6565],
                        [-0.2780, -0.2700, -1.9849, -0.3109],
                        [ 0.2164, -0.5135, -1.0767,  2.2482]])

In [7]:
%timeit t.a = new_value

81.4 ns ± 0.563 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0421, -1.0498,  0.2443],
               [-1.6149, -0.4498, -0.3010]]),
    x: Batch(
           c: tensor([[-0.5066,  0.2429, -0.2976,  0.6565],
                      [-0.2780, -0.2700, -1.9849, -0.3109],
                      [ 0.2164, -0.5135, -1.0767,  2.2482]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0421, -1.0498,  0.2443],
        [-1.6149, -0.4498, -0.3010]])

In [11]:
%timeit b.a

65.1 ns ± 1.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7513, -0.0534,  2.4228],
               [ 0.5126, -0.3575,  0.3762]]),
    x: Batch(
           c: tensor([[-0.5066,  0.2429, -0.2976,  0.6565],
                      [-0.2780, -0.2700, -1.9849, -0.3109],
                      [ 0.2164, -0.5135, -1.0767,  2.2482]]),
       ),
)

In [13]:
%timeit b.a = new_value

606 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1 µs ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.2 µs ± 80.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

303 µs ± 3.02 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

293 µs ± 6.95 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f48e0f52220>
├── 'a' --> tensor([[[ 0.0421, -1.0498,  0.2443],
│                    [-1.6149, -0.4498, -0.3010]],
│           
│                   [[ 0.0421, -1.0498,  0.2443],
│                    [-1.6149, -0.4498, -0.3010]],
│           
│                   [[ 0.0421, -1.0498,  0.2443],
│                    [-1.6149, -0.4498, -0.3010]],
│           
│                   [[ 0.0421, -1.0498,  0.2443],
│                    [-1.6149, -0.4498, -0.3010]],
│           
│                   [[ 0.0421, -1.0498,  0.2443],
│                    [-1.6149, -0.4498, -0.3010]],
│           
│                   [[ 0.0421, -1.0498,  0.2443],
│                    [-1.6149, -0.4498, -0.3010]],
│           
│                   [[ 0.0421, -1.0498,  0.2443],
│                    [-1.6149, -0.4498, -0.3010]],
│           
│                   [[ 0.0421, -1.0498,  0.2443],
│                    [-1.6149, -0.4498, -0.3010]]])
└── 'x' --> <FastTreeValue 0x7f48e0f52040>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.4 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f482b240f40>
├── 'a' --> tensor([[ 0.0421, -1.0498,  0.2443],
│                   [-1.6149, -0.4498, -0.3010],
│                   [ 0.0421, -1.0498,  0.2443],
│                   [-1.6149, -0.4498, -0.3010],
│                   [ 0.0421, -1.0498,  0.2443],
│                   [-1.6149, -0.4498, -0.3010],
│                   [ 0.0421, -1.0498,  0.2443],
│                   [-1.6149, -0.4498, -0.3010],
│                   [ 0.0421, -1.0498,  0.2443],
│                   [-1.6149, -0.4498, -0.3010],
│                   [ 0.0421, -1.0498,  0.2443],
│                   [-1.6149, -0.4498, -0.3010],
│                   [ 0.0421, -1.0498,  0.2443],
│                   [-1.6149, -0.4498, -0.3010],
│                   [ 0.0421, -1.0498,  0.2443],
│                   [-1.6149, -0.4498, -0.3010]])
└── 'x' --> <FastTreeValue 0x7f48e0fb9d60>
    └── 'c' --> tensor([[-0.5066,  0.2429, -0.2976,  0.6565],
                        [-0.2780, -0.2700, -1.9849, -0.3109],
                 

In [23]:
%timeit t_cat(trees)

44.2 µs ± 633 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

83.3 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.5066,  0.2429, -0.2976,  0.6565],
                       [-0.2780, -0.2700, -1.9849, -0.3109],
                       [ 0.2164, -0.5135, -1.0767,  2.2482]],
              
                      [[-0.5066,  0.2429, -0.2976,  0.6565],
                       [-0.2780, -0.2700, -1.9849, -0.3109],
                       [ 0.2164, -0.5135, -1.0767,  2.2482]],
              
                      [[-0.5066,  0.2429, -0.2976,  0.6565],
                       [-0.2780, -0.2700, -1.9849, -0.3109],
                       [ 0.2164, -0.5135, -1.0767,  2.2482]],
              
                      [[-0.5066,  0.2429, -0.2976,  0.6565],
                       [-0.2780, -0.2700, -1.9849, -0.3109],
                       [ 0.2164, -0.5135, -1.0767,  2.2482]],
              
                      [[-0.5066,  0.2429, -0.2976,  0.6565],
                       [-0.2780, -0.2700, -1.9849, -0.3109],
                       [ 0.2164, -0.5135, -1.0767,  2.2482]],

In [26]:
%timeit Batch.stack(batches)

104 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.5066,  0.2429, -0.2976,  0.6565],
                      [-0.2780, -0.2700, -1.9849, -0.3109],
                      [ 0.2164, -0.5135, -1.0767,  2.2482],
                      [-0.5066,  0.2429, -0.2976,  0.6565],
                      [-0.2780, -0.2700, -1.9849, -0.3109],
                      [ 0.2164, -0.5135, -1.0767,  2.2482],
                      [-0.5066,  0.2429, -0.2976,  0.6565],
                      [-0.2780, -0.2700, -1.9849, -0.3109],
                      [ 0.2164, -0.5135, -1.0767,  2.2482],
                      [-0.5066,  0.2429, -0.2976,  0.6565],
                      [-0.2780, -0.2700, -1.9849, -0.3109],
                      [ 0.2164, -0.5135, -1.0767,  2.2482],
                      [-0.5066,  0.2429, -0.2976,  0.6565],
                      [-0.2780, -0.2700, -1.9849, -0.3109],
                      [ 0.2164, -0.5135, -1.0767,  2.2482],
                      [-0.5066,  0.2429, -0.2976,  0.6565],
                   

In [28]:
%timeit Batch.cat(batches)

186 µs ± 2.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

627 µs ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
